In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 3.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/공간정보활용 공모전/

/content/drive/.shortcut-targets-by-id/12CdYaiaXminsUG-MrYvqDyVBDHcUxd6B/공간정보활용 공모전


In [ ]:
from dataProcssingFunction import polygon2gray
from torchvision import transforms

import torch
import numpy as np
import pandas as pd
import geopandas as gpd

print(gpd.__version__)

0.12.2


In [ ]:
file = '/content/drive/MyDrive/공간정보활용 공모전/yangpyeong.csv'
eval = pd.read_csv(file, encoding='cp949')

file = '/content/drive/MyDrive/공간정보활용 공모전/양평군/AL_41830_D194_20221115.shp'
yangpyeong = gpd.read_file(file, encoding='euckr')

In [ ]:
yangpyeong['A2'] = yangpyeong['A2'].astype('int64')
yangpyeong['A1'] = yangpyeong['A1'].astype('int64')
df = pd.merge(eval[eval['기준연도']==2022][['고유번호', '지형형상', '지형형상코드']], yangpyeong[['A1', 'A21', 'A22', 'geometry']], how='inner', left_on='고유번호', right_on='A1')
df = df.drop(['지형형상', 'A1', 'A21', 'A22'], axis=1)
df.rename(columns = {'고유번호' : 'id', '지형형상코드' : 'label'}, inplace = True)
eval= df

In [ ]:
eval

,id,label,geometry
0,4183025021100190000,5,"POLYGON ((243667.051 444549.502, 243667.385 44..."
1,4183025021100250002,5,"POLYGON ((243549.876 444587.228, 243550.293 44..."
2,4183025021100430000,5,"POLYGON ((243558.737 444373.079, 243546.649 44..."
3,4183025021100520006,5,"POLYGON ((243368.424 444209.815, 243367.309 44..."
4,4183025021100630000,5,"POLYGON ((243211.299 444263.362, 243212.952 44..."
...,...,...,...
3362,4183041033103530001,4,"POLYGON ((247510.734 435249.729, 247504.584 43..."
3363,4183041033103770001,4,"POLYGON ((247873.815 435739.637, 247877.735 43..."
3364,4183041033104200001,1,"POLYGON ((247495.839 435539.442, 247492.199 43..."
3365,4183041033200270000,5,"POLYGON ((248196.809 435113.835, 248195.539 43..."


In [ ]:
yangpyeong.dropna(subset=['A21'], inplace=True)
yangpyeong.drop(yangpyeong[yangpyeong.A22.eq('지정되지않음')].index, inplace=True)
yangpyeong.drop(yangpyeong[yangpyeong.A18.eq('도로등')].index, inplace=True)
yangpyeong.drop(yangpyeong[yangpyeong.A18.eq('하천등')].index, inplace=True)
yangpyeong.drop(yangpyeong[yangpyeong.A11.eq('하천')].index, inplace=True)
yangpyeong.drop(yangpyeong[yangpyeong.geometry.type == 'MultiPolygon'].index, inplace=True)

In [ ]:
yangpyeong['A21'].count()

234921

In [ ]:
yangpyeong['A21'].value_counts().sort_index()

01      2684
02      3824
03      9654
04     65094
05    151051
06      2614
Name: A21, dtype: int64

In [ ]:
yangpyeong['A21'].value_counts(normalize=True).sort_index()*100

01     1.142512
02     1.627781
03     4.109467
04    27.708889
05    64.298637
06     1.112714
Name: A21, dtype: float64

In [ ]:
#temp = pd.merge(eval, yangpyeong, how='outer', indicator=True)
#temp = temp.query('_merge == "right_only"').drop(columns=['_merge', 'id', 'label'])
temp = yangpyeong

In [ ]:
temp

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A18,A19,A20,A21,A22,A23,A24,A25,A26,geometry
0,472381282,4183025021100010000,4183025021,경기도 양평군 양평읍 양근리,1,일반,1,1전,2022,01,...,전,03,완경사,5,부정형,12,맹지,203300.0,2022-11-08,"POLYGON ((243572.696 444754.136, 243552.607 44..."
2,472468128,4183025021100020002,4183025021,경기도 양평군 양평읍 양근리,1,일반,2-2,2-2창,2022,01,...,주거기타,03,완경사,5,부정형,08,세로한면(가),407500.0,2022-11-08,"POLYGON ((243594.681 444632.861, 243597.473 44..."
3,472384090,4183025021100020003,4183025021,경기도 양평군 양평읍 양근리,1,일반,2-3,2-3임,2022,01,...,자연림,03,완경사,5,부정형,12,맹지,51000.0,2022-11-08,"POLYGON ((243508.079 444706.674, 243513.364 44..."
4,472407997,4183025021100020004,4183025021,경기도 양평군 양평읍 양근리,1,일반,2-4,2-4임,2022,01,...,전,03,완경사,5,부정형,12,맹지,186900.0,2022-11-08,"POLYGON ((243572.907 444761.590, 243587.679 44..."
5,472441333,4183025021100020005,4183025021,경기도 양평군 양평읍 양근리,1,일반,2-5,2-5임,2022,01,...,전기타,03,완경사,5,부정형,12,맹지,186900.0,2022-11-08,"POLYGON ((243617.495 444685.045, 243593.633 44..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331568,472172632,4183041033200430000,4183041033,경기도 양평군 개군면 상자포리,2,산,43,산43임,2022,01,...,조림,04,급경사,5,부정형,12,맹지,5470.0,2022-11-08,"POLYGON ((248165.325 434368.307, 248165.620 43..."
331569,472156581,4183041033200440000,4183041033,경기도 양평군 개군면 상자포리,2,산,44,산44임,2022,01,...,자연림,04,급경사,5,부정형,12,맹지,8710.0,2022-11-08,"POLYGON ((248165.620 434391.068, 248165.325 43..."
331570,472169561,4183041033200450001,4183041033,경기도 양평군 개군면 상자포리,2,산,45-1,산45-1임,2022,01,...,자연림,04,급경사,5,부정형,12,맹지,8520.0,2022-11-08,"POLYGON ((248025.182 434753.340, 248044.082 43..."
331571,472453123,4183041033200460000,4183041033,경기도 양평군 개군면 상자포리,2,산,46,산46임,2022,01,...,자연림,03,완경사,5,부정형,06,소로한면,16300.0,2022-11-08,"POLYGON ((247729.197 434700.291, 247730.517 43..."


In [ ]:
ratio = eval[eval['기준연도']==2022]['지형형상코드'].value_counts(normalize=True).sort_index().values

In [ ]:
eval[eval['기준연도']==2022]['지형형상코드'].value_counts(normalize=True).sort_index().values

array([0.01718009, 0.01895735, 0.05835308, 0.36433649, 0.53643365,
       0.00473934])

In [ ]:
temp.A21 = temp.A21.astype('int64')
sample = pd.DataFrame([])
for i in range(6):
  sample = pd.concat([sample, temp[temp['A21']==i+1].sample(n=round(50000*ratio[i]), replace=False)])
  print(len(sample))
sample = sample.sample(n=50000, replace=False).reset_index(drop=True)
print(len(sample))

859
1807
4725
22942
49764
50001
50000


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
convert_tensor = transforms.ToTensor()
data = []
cnt = 0
for polygon in sample.geometry:
    arr = polygon2gray(polygon)
    data.append(convert_tensor(arr))
    cnt += 1
    if cnt%10000==0:
      print(cnt)
X = torch.cat(data, 0)

In [ ]:
Y = torch.from_numpy(sample.A21.to_numpy()).long().to(device)

In [ ]:
#Tensor를 numpy로 변경
#X_np = X.numpy( )
#이 때, tensor의 device type이 'cuda'이면 'cpu'로 바꿔줘야 한다.
X_np = X.cpu( ).numpy( )
Y_np = Y.cpu( ).numpy( )


#npy파일로 local에 저장
#numpy 배열 'b'를 경로에 저장
X_file = '/content/drive/MyDrive/공간정보활용 공모전/양평군/train_X_npy_file'
Y_file = '/content/drive/MyDrive/공간정보활용 공모전/양평군/train_Y_npy_file'
np.save(X_file, X_np)
np.save(Y_file, Y_np)

In [ ]:
convert_tensor = transforms.ToTensor()
data = []
cnt = 0
for polygon in eval.geometry:
    arr = polygon2gray(polygon)
    data.append(convert_tensor(arr))
    cnt += 1
    if cnt%10000==0:
      print(cnt)
X = torch.cat(data, 0)

In [ ]:
Y = torch.from_numpy(eval.label.to_numpy()).long().to(device)

In [ ]:
#Tensor를 numpy로 변경
#X_np = X.numpy( )
#이 때, tensor의 device type이 'cuda'이면 'cpu'로 바꿔줘야 한다.
X_np = X.cpu( ).numpy( )
Y_np = Y.cpu( ).numpy( )


#npy파일로 local에 저장
#numpy 배열 'b'를 경로에 저장
X_file = '/content/drive/MyDrive/공간정보활용 공모전/양평군/test_X_npy_file'
Y_file = '/content/drive/MyDrive/공간정보활용 공모전/양평군/test_Y_npy_file'
np.save(X_file, X_np)
np.save(Y_file, Y_np)